In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
titanic = sns.load_dataset('titanic')
iris = sns.load_dataset('iris')
mpg = sns.load_dataset('mpg')



# 1. Iris


a. 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width),
꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등
기초통계량(describe())을 구하시오.

In [3]:
iris.groupby('species').describe()

sepal_length                                              \
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

b. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다.
즉,
Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터
Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터
이 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

In [5]:
#평균
def cal_mean(s):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    low = q1 - 1.5 * (q3 - q1)
    high = q3 + 1.5 * (q3 - q1)

    c = (s>=low) & (s<=high)
    return s[c].mean()

for col in iris[['sepal_length','sepal_width','petal_width','petal_length']]:
    print(col, cal_mean(iris[col]).round(2))

sepal_length 5.84
sepal_width 3.04
petal_width 1.2
petal_length 3.76


In [6]:
#표준편차
def cal_std(s):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    low = q1 - 1.5 * (q3 - q1)
    high = q3 + 1.5 * (q3 - q1)

    c = (s>=low) & (s<=high)
    return s[c].std()

for col in iris[['sepal_length','sepal_width','petal_width','petal_length']]:
    print(col, cal_std(iris[col]).round(2))

sepal_length 0.83
sepal_width 0.4
petal_width 0.76
petal_length 1.77


In [4]:
# 강사님풀이
s = iris[iris.species == 'setosa']['sepal_width']
q1 = s.quantile(0.25) # 판다스를 활용함
q3 = np.percentile(s.values, 75) # 넘파이를 활용

iqr = q3 - q1
lower_limit = q1 - 1.5 * iqr
upper_limit = q3 + 1.5 * iqr

s[s < lower_limit] = np.nan
s[s > upper_limit] = np.nan

s.isna().sum()

s.mean(), s.std()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


(3.43125, 0.32034306743094004)

In [8]:
def get_new_stat(s):
    q1, q3 = s.quantile(0.25), np.percentile(s.values, 75)
    iqr = q3 - q1
    lower_limit = q1 - 1.5 * iqr
    upper_limit = q3 + 1.5 * iqr
    s[(s < lower_limit) | (s > upper_limit)] = np.nan
    return s.isna().sum(), s.mean(), s.std()


In [9]:
for species in iris.species.unique():
  for feature in iris.columns[:-1]:
    s = iris[iris.species == species][feature]
    _, new_mean, new_std = get_new_stat(s)
    print(species, feature, ":", new_mean, new_std)

setosa sepal_length : 5.006 0.35248968721345136
setosa sepal_width : 3.43125 0.32034306743094004
setosa petal_length : 1.4608695652173913 0.12906201931985212
setosa petal_width : 0.2333333333333333 0.08588298157855662
versicolor sepal_length : 5.936 0.5161711470638634
versicolor sepal_width : 2.7700000000000005 0.3137983233784114
versicolor petal_length : 4.285714285714285 0.43779751788545657
versicolor petal_width : 1.3259999999999998 0.19775268000454405
virginica sepal_length : 6.6224489795918355 0.593459246748267
virginica sepal_width : 2.9553191489361703 0.2602745295112517
virginica petal_length : 5.5520000000000005 0.5518946956639834
virginica petal_width : 2.0260000000000002 0.2746500556366674


In [11]:
line = []
for species in iris.species.unique():
  for feature in iris.columns[:-1]:
    s = iris[iris.species == species][feature]
    outlier, new_mean, new_std = get_new_stat(s)
    d = {'species' : species, 'feature' : feature, 'outlier 갯수' : outlier, 'mean' : new_mean, 'std' : new_std}
    line.append(d)

In [15]:
df = pd.DataFrame(line)
df.set_index(['species', 'feature'], inplace = True)
df

outlier 갯수      mean       std
species    feature                                     
setosa     sepal_length           0  5.006000  0.352490
           sepal_width            2  3.431250  0.320343
           petal_length           4  1.460870  0.129062
           petal_width            2  0.233333  0.085883
versicolor sepal_length           0  5.936000  0.516171
           sepal_width            0  2.770000  0.313798
           petal_length           1  4.285714  0.437798
           petal_width            0  1.326000  0.197753
virginica  sepal_length           1  6.622449  0.593459
           sepal_width            3  2.955319  0.260275
           petal_length           0  5.552000  0.551895
           petal_width            0  2.026000  0.274650

In [24]:
# versicolor 만 가져와보기
df.loc['versicolor'] # df.iloc[4:8] # 틀렸으..

# setosa의 sepal_width 만 가져와보기
df.loc[[('setosa','sepal_width')]]

,,outlier 갯수,mean,std
species,feature,,,
setosa,sepal_width,2,3.43125,0.320343


# 2. Titanic

a. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오.
category1 카테고리는 다음과 같이 정의됨
1) 20살이 넘으면 성별을 그대로 사용한다.
2) 20살 미만이면 성별에 관계없이 “child”라고 한다.

In [ ]:
category1 = titanic[['age','sex']]
category1['sex1'] = category1.apply(lambda x: 'child' if x.age < 20 else x.sex ,axis=1)
category1.tail()

In [33]:
# 강사님 풀이
titanic['category1'] = titanic.apply(lambda x : x.sex if x.age >= 20 else 'child' ,axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,24.0,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


b. 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값
이 되도록 titanic 데이터프레임을 고치시오.

In [28]:
# 평균 나이값 구하기
most_age = titanic['age'].value_counts(dropna=True).idxmax()
most_age

24.0

In [ ]:
# 채워졌는지 확인
titanic['age'].fillna(most_age, inplace=True)
titanic.isnull().sum()

In [ ]:
titanic.head(20)
# index 5, 17, 19번 변경됨 확인 완료.

In [36]:
# 강사님 풀이
titanic.age.fillna(titanic.age.mean(), inplace=True)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,24.0,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


c. 성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

In [39]:
df = titanic.loc[:, ['sex', 'class', 'embark_town', 'survived']]
df

,sex,class,embark_town,survived
0,male,Third,Southampton,0
1,female,First,Cherbourg,1
2,female,Third,Southampton,1
3,female,First,Southampton,1
4,male,Third,Southampton,0
...,...,...,...,...
886,male,Second,Southampton,0
887,female,First,Southampton,1
888,female,Third,Southampton,0
889,male,First,Cherbourg,1


In [48]:
# 성별, 선실별 생존율
grouped = df.groupby(['class', 'sex'])
average = grouped.mean()
print(average)

               survived
class  sex             
First  female  0.968085
       male    0.368852
Second female  0.921053
       male    0.157407
Third  female  0.500000
       male    0.135447


In [45]:
# 출발지별 생존율
grouped = df.groupby(['embark_town'])
average = grouped.mean()
print(average)

titanic.groupby('embark_town')[['survived']].mean()

             survived
embark_town          
Cherbourg    0.553571
Queenstown   0.389610
Southampton  0.336957


,survived
embark_town,
Cherbourg,0.553571
Queenstown,0.389610
Southampton,0.336957


In [46]:
# 강사님풀이
titanic.groupby('sex')[['survived']].mean() # 데이터프레임 형태 = [['survived']], 씨리얼 형태 = ['survived']
titanic.groupby('class')[['survived']].mean()

titanic.pivot_table('survived', 'sex', 'class') # 피봇테이블로 만들기

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


d. 타이타닉호 승객을 ‘미성년자’, ‘청년’, ‘중년’, ‘장년’, ‘노년’ 나이 그룹으로 나누고,
각 그룹별 생존율을 구하시오.
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]

In [ ]:
# 그룹 만들기
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]

titanic["agegroup"]=pd.cut(titanic['age'],bins,right=False,labels=labels)
titanic.head()

In [16]:
# 나이 그룹별 생존율
df2 = titanic.loc[:, ['survived','agegroup']]
grouped2 = df2.groupby(['agegroup'])
average = grouped2.mean()
print(average)

          survived
agegroup          
미성년자      0.458599
청년        0.324937
중년        0.417969
장년        0.388060
노년        0.142857


In [50]:
# 강사님풀이
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
titanic["agegroup"]=pd.cut(titanic['age'],bins,labels=labels)

In [51]:
titanic.agegroup.value_counts()

청년      407
중년      241
미성년자    165
장년       59
노년        5
Name: agegroup, dtype: int64

In [55]:
titanic.pivot_table('survived', 'agegroup')
titanic.groupby('agegroup')[['survived']].mean()

,survived
agegroup,
미성년자,0.424242
청년,0.334152
중년,0.423237
장년,0.355932
노년,0.200000


e. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

In [73]:
# 강사님풀이
titanic['age_group'] = pd.qcut(titanic.age, 3, labels = ['A', 'B', 'C'])
titanic.groupby(['age_group', 'sex'])[['survived']].mean()


survived
age_group sex             
A         female  0.711765
          male    0.172535
B         female  0.693878
          male    0.215686
C         female  0.821053
          male    0.198953

# 3. Mile Per Gallon

a. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.

In [82]:
mpg['hp_per_cc'] = mpg['horsepower'] / mpg.displacement # 두가지방법

b. name으로부터 manufacturer(제조사)와 모델을 추출하여
새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.

In [ ]:
mpg['manufacture'] = mpg.name.apply(lambda x : x.split()[0])
mpg['model'] = mpg.name.apply(lambda x : " ".join(x.split()[1:]))
mpg.drop(columns = ['name'], inplace = True)

In [98]:
mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacture,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino
...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,0.614286,ford,mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,0.536082,vw,pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,0.622222,dodge,rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,0.658333,ford,ranger


c. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.

In [88]:
mpg['cylinders'].unique()

array([8, 4, 6, 3, 5])

In [85]:
mpg.groupby('cylinders')[['mpg']].mean()

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [93]:
mpg.pivot_table('mpg', 'cylinders')

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


d. 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.

In [89]:
mpg['origin'].unique()

array(['usa', 'japan', 'europe'], dtype=object)

In [92]:
mpg.groupby(['origin','mpg'])[['hp_per_cc']].mean()

hp_per_cc
origin mpg            
europe 16.2   0.815951
       16.5   0.714286
       17.0   0.766871
       18.0   0.925620
       19.0   0.829477
...                ...
usa    35.7   0.816327
       36.0   0.668254
       36.1   0.673469
       38.0   0.462214
       39.0   0.744186

[178 rows x 1 columns]

In [97]:
mpg.pivot_table('hp_per_cc', ['origin','mpg'])

hp_per_cc
origin mpg            
europe 16.2   0.815951
       16.5   0.714286
       17.0   0.766871
       18.0   0.925620
       19.0   0.829477
...                ...
usa    35.7   0.816327
       36.0   0.668254
       36.1   0.673469
       38.0   0.462214
       39.0   0.744186

[175 rows x 1 columns]

e. 모델이 5개 이상인 제조사에 대하여
연비(mpg)의 평균이 가장 좋은 제조사 Top 5를 구하시오.

In [ ]:
s = mpg.manufacture.value_counts()
s[s >= 5]

In [ ]:
type(big)
big = s[s >= 5].index.tolist()
type(big)

In [118]:
# mpg 데이터에서 big에 속하는것만 나오도록 필터링
# mpg[조건]
# 조건 = mpg.manufature.isin(big)

big_mpg = mpg[mpg.manufacture.isin(big)]
big_mpg.pivot_table('mpg', 'manufacture').sort_values(by = 'mpg', ascending = False).head()

,mpg
manufacture,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000


In [120]:
# 참고
new_mpg = mpg.groupby('manufacture').filter(lambda x : x.manufacture.count() >= 5)

In [124]:
new_mpg.pivot_table('mpg', 'manufacture').sort_values(by= 'mpg', ascending = False).head()

,mpg
manufacture,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000
